<a href="https://colab.research.google.com/github/Krishnaveni15082001/ChatBot/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk

In [2]:
pip install newspaper3k

     |████████████████████████████████| 215kB 10.2MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 7.4MB 12.3MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=28704b55029035f6c7c14b385aea8c55fef294db0c437ef4ad0df154f4bb9c07
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13538 sha256=6fafe03d2d2c49cd8bd1718e685a00d37455f6823f09efb5db755e9e36e27aec
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=dcff55ffd38c29430245fb67837a8b64ceffd1f23c5c6ba41fea6523dbe96e67
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c

In [8]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [9]:
#import libraries
from newspaper import Article

import random
import string
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import pandas as pd 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from IPython import display
from nltk.corpus import PlaintextCorpusReader
warnings.filterwarnings('ignore')
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
nltk.download('punkt',quiet=True)

True

In [11]:
#pre processing data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        sentence_list = nltk.word_tokenize(pattern)
        words.extend(sentence_list)
        #add documents in the corpus
        documents.append((sentence_list, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [12]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(sentence_list.lower()) for sentence_list in words if sentence_list not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [13]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for sentence_list in words:
        bag.append(1) if sentence_list in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [21]:
article = Article('https://www.nationalgeographic.com/environment/article/global-warming-overview')
article.download()
article.parse()
article.nlp()
corpus = article.text

In [22]:
text = corpus
sentence_list = nltk.sent_tokenize(text) 

In [23]:
def index_sort(list_var):
  length = len(list_var)
  list_index = list(range(0,length))
  x = list_var
  for i in range(length):
    for j in range(length):
      if x[list_index[i]] > x[list_index[j]]:
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp
  return list_index

In [24]:
def greetings(text):
  text = text.lower()
  bot_greeting = ['hi','hello','howdy','hola','wassup']
  user_greeting = ['hi','hello','hey','hola']
  for word in text.split():
    if word in user_greeting:
      return random.choice(bot_greeting)

In [25]:
def output(text):
    exit_list = ['exit','bye','quit','break','see you later']
    while True:
      user_input = text
      if user_input.lower() in exit_list:
        return 'bye'
        break
      else:
        if greetings(user_input) != None:
          text = text.lower()
          bot_greeting = ['hi','hello','howdy','hola','wassup']
          user_greeting = ['hi','hello','hey','hola']
          for word in text.split():
            if word in user_greeting:
              return random.choice(bot_greeting)
        else:
              user_input = user_input.lower()
              sentence_list.append(user_input)
              bot_response = ''
              cm = CountVectorizer().fit_transform(sentence_list)
              similarity_scores = cosine_similarity(cm[-1], cm)
              similarity_scores_list = similarity_scores.flatten()
              index = index_sort(similarity_scores_list)
              index = index[1:]
              response_flag = 0
              j=0
              for i in range(len(index)):
                if similarity_scores_list[index[i]] > 0.0:
                  bot_response = bot_response+' '+sentence_list[index[i]]
                  response_flag = 1
                  j = j+1
                  if j>2:
                    break
              if response_flag == 0:
                bot_response = bot_response+' '+"I apologize, I don't understand."
              sentence_list.remove(user_input)
              return bot_response

In [26]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
10/10 [==============================] - 14s 2ms/step - loss: 2.3139 - accuracy: 0.0439 
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1757 - accuracy: 0.2051
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1016 - accuracy: 0.1623
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1297 - accuracy: 0.2709
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9826 - accuracy: 0.3071
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8238 - accuracy: 0.4959
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8501 - accuracy: 0.3451
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6058 - accuracy: 0.4213
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7103 - accuracy: 0.4853
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4825 - accuracy: 0.5341
Epoch 1

In [27]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [29]:
print("Hi how can i help you?")

exit_list = ['exit','bye','quit','break','see you later']

while True:
  user_input = input()
  if user_input.lower() in exit_list:
    print('bye')
    break
  else:
    if greetings(user_input) != None:
      print('Bot :'+greetings(user_input))
    else:
      print('Bot :'+output(user_input))

Hi how can i help you?
hi
Bot :hello
what is global warming?
Bot : What will we do—what can we do—to slow this human-caused warming? The short timescale of this recent warming is singular as well. This natural greenhouse effect is what keeps the Earth's climate livable.
what is greenhouse effect
Bot : This natural greenhouse effect is what keeps the Earth's climate livable. Photograph by Paul Nicklen, Nat Geo Image Collection

Understanding the greenhouse effect

The "greenhouse effect" is the warming that happens when certain gases in Earth's atmosphere trap heat. In 1895, the Swedish chemist Svante Arrhenius discovered that humans could enhance the greenhouse effect by making carbon dioxide, a greenhouse gas.
bye
bye
